# Mintpy Time Series
George Brencher

In [ ]:
# Clone MintPy, create environment, install MintPy 
# git clone https://github.com/insarlab/MintPy.git
# mamba env create -f MintPy/docs/environment.yml
# conda activate mintpy
# python -m ipykernel install --user --name=mintpy

In [12]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import mintpy
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network
from mintpy.unwrap_error_phase_closure import plot_num_triplet_with_nonzero_integer_ambiguity

In [13]:
# define work directory
work_dir = os.path.expanduser('/tmp/mintpy')
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

Go to work directory: /tmp/mintpy


In [14]:
# Function to write to config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process ISCE products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [5]:
# View MintPy options
!smallbaselineApp.py --help

usage: smallbaselineApp.py [-h] [--dir WORKDIR] [-g] [-H] [-v] [--plot]
                           [--start STEP] [--end STEP] [--dostep STEP]
                           [customTemplateFile]

Routine Time Series Analysis for Small Baseline InSAR Stack

positional arguments:
  customTemplateFile    custom template with option settings.
                        ignored if the default smallbaselineApp.cfg is input.

optional arguments:
  -h, --help            show this help message and exit
  --dir WORKDIR, --work-dir WORKDIR
                        work directory, (default: ./).
  -g                    generate default template (if it does not exist) and exit.
  -H                    print the default template file and exit.
  -v, --version         print software version and exit
  --plot                plot results [only] without running smallbaselineApp.

steps processing (start/end/dostep):
  Command line options for steps processing with names are chosen from the following list:
  
  

In [19]:
# Write config file for MintPy. For a list of processing options, see: 
#https://nbviewer.org/github/insarlab/MintPy-tutorial/blob/main/smallbaselineApp.ipynb
CONFIG_TXT = '''# vim: set filetype=cfg:
##----------------------------- SentinelStack/ISCE ---------------------##
#ssaraopt = --platform=SENTINEL-1A,SENTINEL-1B -f 84  -s 2021-08-01, -e 2021-09-01
#processor                       = isce
#sentinelStack.demDir            = ../DEM
#sentinelStack.boundingBox       = '27 28 86.3 87'
#sentinelStack.subswath          = 1 2  # comment 
#sentinelStack.numConnections    = 2   # comment
#sentinelStack.azimuthLooks      = 2   # comment
#sentinelStack.rangeLooks        = 9  # comment
#sentinelStack.filtStrength      = 0.2  # comment
#sentinelStack.unwMethod         = snaphu  # comment
#sentinelStack.coregistration    = auto  # comment

##-------------------------------- MintPy -----------------------------##
mintpy.load.processor        = isce
##---------for ISCE only:
mintpy.load.metaFile         = ../reference/IW*.xml
mintpy.load.baselineDir      = ../baselines
##---------interferogram datasets:
mintpy.load.unwFile          = ../merged/interferograms/*/filt_*.unw
mintpy.load.corFile          = ../merged/interferograms/*/filt_*.cor
mintpy.load.connCompFile     = ../merged/interferograms/*/filt_*.unw.conncomp
##---------geometry datasets:
mintpy.load.demFile          = ../merged/geom_reference/hgt.rdr
mintpy.load.lookupYFile      = ../merged/geom_reference/lat.rdr
mintpy.load.lookupXFile      = ../merged/geom_reference/lon.rdr
mintpy.load.incAngleFile     = ../merged/geom_reference/los.rdr
mintpy.load.azAngleFile      = ../merged/geom_reference/los.rdr
mintpy.load.shadowMaskFile   = ../merged/geom_reference/shadowMask.rdr
mintpy.load.waterMaskFile    = None
'''

config_file = os.path.join('/tmp/mintpy', "imja_august2021_SenTA12.txt")
write_config_file(config_file, CONFIG_TXT, mode='w')

write configuration to file: /tmp/mintpy/imja_august2021_SenTA12.txt


In [20]:
# Load data into MintPy
!smallbaselineApp.py imja_august2021_SenTA12.txt --dostep load_data

MintPy version v1.3.2, date 2021-11-21
--RUN-at-2022-03-08 16:03:41.058385--
Current directory: /tmp/mintpy
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Project name: imja_august2021_SenTA12
Go to work directory: /tmp/mintpy
copy default template file /srv/conda/envs/mintpy/lib/python3.8/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /tmp/mintpy/imja_august2021_SenTA12.txt
update default template based on input custom template
    mintpy.load.processor: auto --> isce
    mintpy.load.metaFile: auto --> ../reference/IW*.xml
    mintpy.load.baselineDir: auto --> ../baselines
    mintpy

In [18]:
# Examine loaded inputs to MintPy
!ls -l inputs

total 28
-rw-r--r-- 1 jovyan jovyan  1989 Mar  7 03:42 imja_august2021.txt
-rw-r--r-- 1 jovyan jovyan 21654 Mar  7 03:42 smallbaselineApp.cfg


In [8]:
# Plot all data related to one interferometric pair
view.main('./inputs/ifgramStack.h5 20210806_20210818 --ncols 3'.split())

FileNotFoundError: input file file ./inputs/ifgramStack.h5 NOT exist!

In [ ]:
# Examine radar geometry files
view.main('./inputs/geometryRadar.h5'.split())

In [ ]:
# Optional step prior to inversion. Remove interferograms based on coherence, temporal/perpendicular baselines, date, etc. 
!smallbaselineApp.py imja_august2021.txt --dostep modify_network

In [ ]:
# Examine interferogram network 
plot_network.main(['./inputs/ifgramStack.h5'])

In [ ]:
# Select reference point. Should be coherent and close to the aoi. 
!smallbaselineApp.py imja_august2021.txt --dostep reference_point

In [ ]:
# Without inversion, quickly assesses possible deformation and phase unwrapping error
!smallbaselineApp.py imja_august2021.txt --dostep quick_overview

In [ ]:
# Plot results of phase stacking 
view.main('avgPhaseVelocity.h5'.split())

In [ ]:
# Plot number of triplets with nonzero integer ambiguity, related to unwrapping error
plot_num_triplet_with_nonzero_integer_ambiguity('numTriNonzeroIntAmbiguity.h5', display=True, fig_size=[12, 4])

In [ ]:
# Do time series inversion. 
!smallbaselineApp.py imja_august2021.txt --dostep invert_network

In [ ]:
# Plot the raw phase time-series re-wrapped into (-5, 5) cm
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

In [ ]:
# Plot time series temporal coherence and mask coherence 
view.main('temporalCoherence.h5 -c gray --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 -c gray --notick --noaxis --noverbose'.split())

In [ ]:
# Correct for tropospheric correction. Copernicus Climate Data Store (CDS) registration is needed to access ERA5 data. 
# To create account and set up api, follow steps at https://github.com/insarlab/pyaps#2-account-setup-for-era5
!smallbaselineApp.py imja_august2021.txt --dostep correct_troposphere

In [ ]:
# View tropospheric corrected time series
view.main('timeseries_ERA5.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Remove linear or quadratic ramps based on the phase of reliable pixels. Recommended for localized deformation signals
!smallbaselineApp.py imja_august2021.txt --dostep deramp

In [ ]:
# View deramped time series
view.main('timeseries_ERA5_ramp.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Correct phase residual caused by DEM error
!smallbaselineApp.py imja_august2021.txt --dostep correct_topography

In [ ]:
# View DEM error-corrected time series
view.main('timeseries_ERA5_ramp_demErr.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Calculate average velocity of time series 
!smallbaselineApp.py imja_august2021.txt --dostep velocity

In [ ]:
# View original and troposphere corrected velocities
view.main('velocity.h5 --notick --noaxis --noverbose'.split())
view.main('velocityERA5.h5 --notick --noaxis --noverbose'.split())

In [ ]:
# Geocode outputs
!smallbaselineApp.py imja_august2021.txt --dostep geocode

In [ ]:
# View geocoded outputs
!ls -l ./geo

In [ ]:
# Plot geocoded velocity 
view.main('./geo/geo_velocity.h5 velocity --dem ./geo/geo_geometryRadar.h5 --shade-exag 0.05 --figsize 12 12 --notitle --notick --noaxis'.split())